<a href="https://colab.research.google.com/github/Chinmayi-Bachu/Object_detection/blob/main/weapon_detection_yolov11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!nvidia-smi
!pip install ultralytics

Sat Aug  9 04:55:20 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   52C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
import zipfile, os, shutil
from sklearn.model_selection import train_test_split

# Define paths
zip_path = '/content/data.zip'
extract_path = '/content/weapons-dataset'
images_path = os.path.join(extract_path, 'images')
labels_path = os.path.join(extract_path, 'labels')

# Unzip the dataset
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Gather all images and corresponding labels
image_files = [f for f in os.listdir(images_path) if f.endswith(('.jpg', '.png'))]
label_files = [f.replace('.jpg', '.txt').replace('.png', '.txt') for f in image_files]

# Split into train, val, test (70%, 20%, 10%)
train_imgs, test_imgs = train_test_split(image_files, test_size=0.3, random_state=42)
val_imgs, test_imgs = train_test_split(test_imgs, test_size=1/3, random_state=42)

# Helper function to copy files
def copy_files(img_list, dest_img_dir, dest_label_dir):
    os.makedirs(dest_img_dir, exist_ok=True)
    os.makedirs(dest_label_dir, exist_ok=True)
    for img_name in img_list:
        label_name = img_name.replace('.jpg', '.txt').replace('.png', '.txt')
        shutil.copy(os.path.join(images_path, img_name), dest_img_dir)
        shutil.copy(os.path.join(labels_path, label_name), dest_label_dir)

# Organize directories
base = '/content/yolo_gun_dataset'
copy_files(train_imgs, f'{base}/train/images', f'{base}/train/labels')
copy_files(val_imgs, f'{base}/val/images', f'{base}/val/labels')
copy_files(test_imgs, f'{base}/test/images', f'{base}/test/labels')

In [4]:
yaml_content = """
path: /content/yolo_gun_dataset
train: train/images
val: val/images
test: test/images

names:
  0: gun
"""

with open('/content/gun_data.yaml', 'w') as f:
    f.write(yaml_content)

In [1]:
from ultralytics import YOLO

# Load a pre-trained YOLOv11 model
# This is the key change from the YOLOv8 code. We use 'yolo11n.pt'
print("Loading YOLOv11 model...")
try:
    model = YOLO('yolo11n.pt')
    print("YOLOv11 model loaded successfully.")
except Exception as e:
    print(f"Error loading YOLOv11 model: {e}. Please ensure the model file exists.")
    # Exit if model fails to load
    exit()

# Train the YOLOv11 model
print("Starting YOLOv11 model training...")
model.train(data='/content/gun_data.yaml', epochs=50, imgsz=640, batch=16)
print("YOLOv11 model training complete.")

# Run inference on the trained YOLOv11 model
print("Running inference on the test set...")
results = model.predict(source='/content/yolo_gun_dataset/test/images', save=True, conf=0.5)
print("Inference complete. Results saved in the 'runs/detect' directory.")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Loading YOLOv11 model...


YOLOv11 model loaded successfully.
Starting YOLOv11 model training...
Ultralytics 8.3.176 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/gun_data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, 

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128, 1, True]           
  7                  -1  1    295424  ultralytics

 23        [16, 19, 22]  1    430867  ultralytics.nn.modules.head.Detect           [1, [64, 128, 256]]           
YOLO11n summary: 181 layers, 2,590,035 parameters, 2,590,019 gradients, 6.4 GFLOPs

Transferred 448/499 items from pretrained weights
Freezing layer 'model.23.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...
AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1165.5±828.4 MB/s, size: 49.1 KB)


train: Scanning /content/yolo_gun_dataset/train/labels... 368 images, 1 backgrounds, 0 corrupt: 100%|██████████| 368/368 [00:00<00:00, 2575.01it/s]

...

      50/50      3.39G       1.28     0.8274      1.389         21        640: 100%|██████████| 23/23 [00:06<00:00,  3.59it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:00<00:00,  4.81it/s]

                   all        105        127      0.856      0.793      0.791      0.452



50 epochs completed in 0.106 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 5.5MB
Optimizer stripped from runs/detect/train/weights/best.pt, 5.5MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.176 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:01<00:00,  2.15it/s]


                   all        105        127      0.834      0.795      0.797      0.452
Speed: 0.2ms preprocess, 2.9ms inference, 0.0ms loss, 4.3ms postprocess per image
Results saved to runs/detect/train
YOLOv11 model training complete.
Running inference on the test set...

image 1/53 /content/yolo_gun_dataset/test/images/002e137e-ASP_111_jpeg.rf.f54cc9a0d8a4adada8a5bf20c1747aba.jpg: 640x640 (no detections), 11.3ms
image 2/53 /content/yolo_gun_dataset/test/images/03a5e1ca-63_jpeg.rf.08da241bab2774385788b743601edbaa.jpg: 640x640 1 gun, 11.1ms
image 3/53 /content/yolo_gun_dataset/test/images/0aa18171-ASP_151_png.rf.17a536703b39c0e5d64b3a7e805ace53.jpg: 640x640 (no detections), 11.3ms
image 4/53 /content/yolo_gun_dataset/test/images/0c775095-298_jpeg.rf.8fad8b5e42c58a876f143a81b25a723e.jpg: 640x640 1 gun, 11.0ms
image 5/53 /content/yolo_gun_dataset/test/images/0d602a4b-105_jpeg.rf.fb2aae2d363fe7b73b57bd590d61a290.jpg: 640x640 1 gun, 11.0ms
image 6/53 /content/yolo_gun_dataset/test/image

In [6]:
# Print the performance metrics
# Access the metrics from the results object
metrics = model.metrics

print("Performance Metrics:")
print(f"mAP50: {metrics.box.map50}")
print(f"mAP50-95: {metrics.box.map}") # This is the mAP across all IoU thresholds from 50 to 95
print(f"Precision: {metrics.box.p}")
print(f"Recall: {metrics.box.r}")

Performance Metrics:
mAP50: 0.7967845218478894
mAP50-95: 0.45225252309919045
Precision: [    0.83413]
Recall: [    0.79528]
